In [ ]:
import os
import pandas as pd

In [ ]:
# 경로 설정
# https://storage.googleapis.com/openimages/web/download_v6.html
ANNOTATIONS_FILE = "oidv6-test-annotations-bbox.csv"
CLASS_DESCRIPTIONS_FILE = "class-descriptions.csv"
type = ANNOTATIONS_FILE.split('-')[1]
OUTPUT_FILE = f'door_{type}_image_list.txt'

# Door 클래스 ID
DOOR_CLASS_ID = "/m/02dgv"

# 데이터 로드
annotations = pd.read_csv(ANNOTATIONS_FILE)
class_descriptions = pd.read_csv(
    CLASS_DESCRIPTIONS_FILE, header=None, names=["LabelName", "Class"]
)

# Door 클래스 필터링
door_annotations = annotations[annotations["LabelName"] == DOOR_CLASS_ID]
image_ids = door_annotations["ImageID"].unique()

# 이미지 목록 저장
with open(OUTPUT_FILE, "w") as f:
    for image_id in image_ids:
        f.write(f"{type}/{image_id}\n")

print(f"Door 클래스 {type}에 해당하는 {len(image_ids)}개의 이미지 ID를 {OUTPUT_FILE}에 저장했습니다.")

Door 클래스 test에 해당하는 546개의 이미지 ID를 door_test_image_list.txt에 저장했습니다.


In [61]:
print(f"python getDoorImage_script.py door_{type}_image_list.txt --download_folder door_images --num_processes 10")

python getDoorImage_script.py door_test_image_list.txt --download_folder door_images --num_processes 10


In [62]:
# 라벨링 데이터와 이미지 연결
# 파일 경로 설정
ANNOTATIONS_FILE = f"oidv6-{type}-annotations-bbox.csv"
CLASS_DESCRIPTIONS_FILE = "class-descriptions.csv"

# Door 클래스 ID
DOOR_CLASS_ID = "/m/02dgv"

# 라벨링 데이터 로드
annotations = pd.read_csv(ANNOTATIONS_FILE)
class_descriptions = pd.read_csv(CLASS_DESCRIPTIONS_FILE, header=None, names=["LabelName", "Class"])

# Door 클래스 라벨링 정보 필터링
door_annotations = annotations[annotations["LabelName"] == DOOR_CLASS_ID]

# # 결과 확인
# print(door_annotations.head())


# 라벨링 데이터와 이미지 결합
# 다운로드한 이미지 경로
IMAGE_DIR = "door_images"
OUTPUT_DIR = "labeled_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Door 클래스 이미지와 라벨링 데이터 결합
for image_id in os.listdir(IMAGE_DIR):
    image_id = image_id.replace(".jpg", "")
    image_annotations = door_annotations[door_annotations["ImageID"] == image_id]
    
    if not image_annotations.empty:
        # 라벨링 데이터 저장
        label_file = os.path.join(OUTPUT_DIR, f"{image_id}.txt")
        with open(label_file, "w") as f:
            for _, row in image_annotations.iterrows():
                bbox = [row["XMin"], row["YMin"], row["XMax"], row["YMax"]]
                f.write(f"{DOOR_CLASS_ID} " + " ".join(map(str, bbox)) + "\n")